In [1]:
import numpy as np
from math import log
from functools import reduce
import operator
import matplotlib.pyplot as plt

In [2]:
'''
[[1, 1, true],
[1, 1, true],
[1, 2, false],
...
]
'''
# 计算信息熵
# 主要是第 k 类样本的信息，本实例中的类别只有两个，true or false

# def _calc(p1, p2):
#     return p1 * log(p1, 2) - p2 * log(p2, 2)


def calc_ent(data_set):
    set_size = len(data_set)
#     data_set[:, -1]
    feat = {}
    # 统计 k 类样本，对应的数量
    for data in data_set:
        label = data[-1]
        feat[label] = feat.get(label, 0) + 1
    ent = 0.0
#     def _c(x, y):
#         x_p = x * 1.0 / set_size
#         y_p = y * 1.0 / set_size
#         print(x_p, y_p)
#         return x_p * log(x_p, 2) - y_p * log(y_p, 2)
    # 信息熵公式
    # 
    for label in feat:
        p    = float(feat[label]) / set_size
        ent -= p * log(p, 2)
    return ent
#     return reduce(lambda x, y : x * 1.0 / set_size - y * 1.0 / set_size, feat.values())
    # insert 影响元数据
#     need_reduce = list(feat.values())
#     need_reduce.insert(0, 0)
#     print(need_reduce)
#     return reduce(_c, need_reduce)


In [3]:
test = [[1, 1, 'yes'],
        [1, 1, 'yes'],
        [1, 0, 'no'],
        [0, 1, 'no'],
        [0, 1, 'no'],
       ]
labels = ['no surface', 'flippers']
calc_ent(test)

0.9709505944546686

In [4]:
test = [[1, 1, 'yes'],
        [1, 1, 'yes'],
        [1, 0, 'no'],
        [0, 1, 'no'],
        [0, 1, 'none'],
       ]
calc_ent(test)

1.5219280948873621

In [5]:
d = [1, 2, 33]

dd = d[:]
dd[0] = 22
d
dd

[22, 2, 33]

由此可见，分类多了，信息越混乱，熵越高

In [6]:
# 属性 - feat 色泽
# 属性值 - feat_value 乌黑
def split_data_set_by_feat(data_set, feat, feat_value):
    split_data_set = []
    for data in data_set:
        if data[feat] == feat_value:
            reduced = data[:feat]
            reduced.extend(data[feat + 1:])
            split_data_set.append(reduced) # 把指定属性剔除了
    return split_data_set


# 找到最好的划分方式
def find_best_feat_split(data_set):
    sample_num = len(data_set)
    base_ent   = calc_ent(data_set)
    feat_num   = len(data_set[0]) - 1 # 属性个数
    max_gain   = -1
    best_feat  = -1
    gain       = 0.0
    for i in range(feat_num):
        feat_values      = [ example[i] for example in data_set ]
        feat_unique_vals = set(feat_values) # 统计属性值有几种
        sub_ent          = 0.0
        gain             = 0.0
        # 根据属性与拥有的属性值进行划分 计算总的信息熵
        for feat_value in feat_unique_vals:
            sub_sample = split_data_set_by_feat(data_set, i, feat_value)
#             print(sub_sample)
            sub_ent   += len(sub_sample) * 1.0 / sample_num * calc_ent(sub_sample)
        # 计算信息增益，找最大的增益的属性划分
        gain = base_ent - sub_ent
        if gain > max_gain:
            max_gain  = gain
            best_feat = i
    return best_feat

In [7]:
test = [[1, 1, 'yes'],
        [1, 1, 'yes'],
        [1, 0, 'no'],
        [0, 1, 'no'],
        [0, 1, 'no'],
       ]
find_best_feat_split(test)

0

In [8]:
# classList, 所有样本的类别值
def majority(class_list):
    c = {}
    for class_ in class_list:
        c[class_] = c.get(class_, 0) + 1
    sorted_count = sorted(c.items(), key = operator.itemgetter(1), reverse = True)
    return sorted_count[0][0]

# 在西瓜书中
def create_tree(data_set, feat_names):
    class_list = [ exa[-1] for exa in data_set ]
    # 递归终止条件
    # 所有样本都属于一个类别 k
    if class_list.count(class_list[0]) == len(class_list):
        return class_list[0] # 返回这个类别
    # 属性集为空
    if len(data_set[0]) == 1:
        return majority(class_list)
    
#     print(data_set)
    best_feat      = find_best_feat_split(data_set)
    best_feat_name = feat_names[best_feat]
    # 样本集中会把对应属性剔除，所以对应的名字也要删除
    del(feat_names[best_feat])
    node          = { best_feat_name : {} }
    feat_values   = set([ exa[best_feat] for exa in data_set ])
    for feat_value in feat_values:
        sub_feat_names = feat_names[:]
        node[best_feat_name][feat_value] = create_tree(split_data_set_by_feat(data_set, best_feat, feat_value), sub_feat_names)
    return node


In [9]:
test = [[1, 1, 'yes'],
        [1, 1, 'yes'],
        [1, 0, 'no'],
        [0, 1, 'no'],
        [0, 1, 'no'],
       ]
labels = ['no surface', 'flippers']
tree = create_tree(test, labels)

In [10]:
# 其实就是 n 叉树的遍历，对于每一个节点遍历，而不是考虑多个。
def classify(input_tree, feats, test_vec):
    # 当前节点代表一种属性划分，而 key 是属性 {'no surface': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
    # { key:value} 一个 dict 代表一个节点，同时 key 代表分支，value 是其指向的元素，可以是节点，也可以是值
#     classify_branchs = input_tree.keys()
    # 当前属性划分
    cur_attr_split   = list(input_tree.keys())[0]
    print(cur_attr_split)
    print(feats)
    cur_attr_index   = feats.index(cur_attr_split.strip())
    classify_branch_nodes = input_tree[cur_attr_split]
    for feat_value in classify_branch_nodes.keys():
        if test_vec[cur_attr_index] == feat_value:
            if type(classify_branch_nodes[feat_value]) == dict:
                return classify(classify_branch_nodes[feat_value], feats, test_vec)
            else:
                return classify_branch_nodes[feat_value]


In [11]:
classify(tree, ['no surface', 'flippers'], [1, 0])

no surface
['no surface', 'flippers']
flippers
['no surface', 'flippers']


'no'

In [12]:
aa = {
    1:1,
    3:3,
    2:33,
}
# aa.items()
aa.values()
sum(map(lambda x : x * 1.0 / 3, aa.values()))
# reduce(lambda x, y : x - y, aa.values())
# reduce(f, [x1, x2, x3, x4]) = f(f(f(x1, x2), x3), x4)
# 37 / 3

12.333333333333334

In [13]:
b = {}
b.get(True, 1)

1

In [14]:
a = np.array([
    [1,2,True],
    [3, 3, True],
    [3, 3, True],
    [3, 3, False],
])

np.sum(a[:,-1] == False)# 最后一行
#a[:][-1] 具体到某一行
# a[1,:]

1

In [28]:
# 准备隐形眼镜数据
with open('./lenses.txt', 'r') as f:
    lines = f.readlines()
    x_train = [ line.strip().split('\t') for line in lines ]
#     class_  = list(set([line.strip().split('\t')[-1] for line in lines]))
#     print(x_train)
feat_names  = ['age', 'prescript', 'astigmatic', 'tear_rate']
lenses_tree = create_tree(data_set = x_train, feat_names = feat_names)
lenses_tree

{'tear_rate': {'normal': {'astigmatic': {'yes': {'prescript': {'hyper': {'age': {'presbyopic': 'no lenses',
        'pre': 'no lenses',
        'young': 'hard'}},
      'myope': 'hard'}},
    'no': {'age': {'presbyopic': {'prescript': {'hyper': 'soft',
        'myope': 'no lenses'}},
      'pre': 'soft',
      'young': 'soft'}}}},
  'reduced': 'no lenses'}}